# Challenge FIAP 2024 | 1TSCPV-2024
## Grupo DataStorm
#### Ana Beatriz Azevedo RM 557420
#### Heloiza Oliveira RM 558881
#### Isabelle Nahas RM 557405
#### Matheus Madrid RM 555799
#### Sara Sitta RM 555113

# Segunda etapa do tratamento dos dados, dentro do Big Query. O objetivo é padronizar todas as tabelas para que tragam apenas os dados diários entre 2022 a Jun-2024, o código da Subprefeitura e os valores numéricos atribuídos para cada variável.

# ETL Interrupção de Energia

## DE-PARA Subestação de Energia para Subprefeitura


*   Não há uma chave pronta que vincule as subestações com as subprefeituras. A solução encontrada foi mapear o lat/long de cada subestação e cruzar com o lat/long de cada subprefeitura, criar um raio de aproximação através do Geopy e assim fazer essa atribuição de forma geográfica. Esse De-Para foi realizado fora do Big Query (documentação entregue com o projeto) e a Tabela final armazenada no Google Drive e transferida para cá 'DF_DEPARA_SUBESTACAO_SUB'.



In [ ]:
import pandas as pd
from pandas_gbq import to_gbq
import bigframes.pandas as bf


In [ ]:
bf.options.bigquery.location = "US"
bf.options.bigquery.project = "elated-drive-432523-s4"

In [ ]:
df = bf.read_gbq("elated-drive-432523-s4.RAW.DF_DEPARA_SUBESTACAO_SUB")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2024-09-15 20:32:25.015467+00:00 to avoid incompatibilies with previous reads of this table. To read the latest version, set `use_cache=False` or close the current session with Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# Remove colunas
df = df.drop(columns=['Unnamed: 0','Endereço','DscConjuntoUnidadeConsumidora','Lat','Long'])

In [ ]:
df_interrupcoes = bf.read_gbq("elated-drive-432523-s4.RAW.DF_INTERRUPCOES")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2024-09-15 20:32:27.043572+00:00 to avoid incompatibilies with previous reads of this table. To read the latest version, set `use_cache=False` or close the current session with Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


## Tratamento de DF_INTERRUPÇOES e merge com DE-PARA de Subestacao e Subprefeitura

In [ ]:
# Remove colunas
df_interrupcoes = df_interrupcoes.drop(columns=['DatGeracaoConjuntoDados','IdeConjuntoUnidadeConsumidora','DscConjuntoUnidadeConsumidora','DscAlimentadorSubestacao','NumOrdemInterrupcao','NumUnidadeConsumidora','NumConsumidorConjunto','NomAgenteRegulado','NumCPFCNPJ'])

In [ ]:
# Mesclando os DataFrames
df_interrupcoes = df_interrupcoes.merge(df, left_on='DscSubestacaoDistribuicao', right_on='DscSubestacaoDistribuicao', how='right')



## Criação do DataFrame final no formato TRUSTED

In [ ]:
# Convertendo a DATA
df_interrupcoes['DatInicioInterrupcao'] = pd.to_datetime(df_interrupcoes['DatInicioInterrupcao'])

In [ ]:
# Atribuindo o valor 1 aos casos de interrupção de energia
df_interrupcoes = df_interrupcoes.assign(INTERRUPCOES=1)

In [ ]:
# Agrupando o DF por Subprefeitura e Data, somando os valores de interrupção de energia
df_interrupcoes_tt = df_interrupcoes.groupby(['subprefeitura_mais_proxima', 'DatInicioInterrupcao'], as_index=False)['INTERRUPCOES'].sum()


In [ ]:
# Renomeando colunas
df_interrupcoes_tt = df_interrupcoes_tt.rename(columns={
    'subprefeitura_mais_proxima': 'SUB',
    'DatInicioInterrupcao': 'DATA'
})

In [ ]:
df_interrupcoes_tt = df_interrupcoes_tt.to_pandas()

In [ ]:
df_interrupcoes_tt

,SUB,DATA,INTERRUPCOES
0,AD,2022-01-02 12:44:48+00:00,1
1,AD,2022-01-03 17:32:23+00:00,1
2,AD,2022-01-04 12:26:02+00:00,2
3,AD,2022-01-05 13:15:29+00:00,1
4,AD,2022-01-05 16:30:55+00:00,2
...,...,...,...
54777,VP,2024-06-28 04:50:41+00:00,1
54778,VP,2024-06-28 04:50:44+00:00,1
54779,VP,2024-06-28 06:33:54+00:00,1
54780,VP,2024-06-28 08:20:58+00:00,1


In [ ]:
# Salvando na camada Trusted

import pandas as pd
from pandas_gbq import to_gbq

project_id = "elated-drive-432523-s4"
table_id = "TRUSTED.DF_INTERRUPCOES_TT"

to_gbq(df_interrupcoes_tt, table_id, project_id=project_id, if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 6141.00it/s]


# ETL Alagamento

In [ ]:
import pandas as pd
import bigframes.pandas as bf
df_alagamento = bf.read_gbq("elated-drive-432523-s4.RAW.DF_ALAGAMENTO")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2024-09-15 21:17:50.180365+00:00 to avoid incompatibilies with previous reads of this table. To read the latest version, set `use_cache=False` or close the current session with Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


## Tratamento de DF_ALAGAMENTO

In [ ]:
# Removendo duplicatas
df_alagamento = df_alagamento.drop_duplicates()

In [ ]:
# Removendo duplicatas
df_alagamento = df_alagamento.drop_duplicates().reset_index(drop=True)

In [ ]:
# Convertendo a DATA
df_alagamento['DATA'] = pd.to_datetime(df_alagamento['DATA'], errors='coerce')

In [ ]:
# Removendo colunas
df_alagamento = df_alagamento.drop(columns=['LOCAL','REF','SENTIDO','long','lat','REFERENCIA'])

## Criação do DataFrame final no formato TRUSTED

In [ ]:
# Atribuindo valor 1 aos casos de Alagamento
df_alagamento = df_alagamento.assign(ALAGAMENTO=1)

In [ ]:
df_alagamento = df_alagamento.to_pandas()

In [ ]:
# Separando os dois tipos de condição de alagamento, transitável e intransitável, e depois concatenando de volta o df

one_hot_encoded = pd.get_dummies(df_alagamento['CONDICAO'])
df_alagamento_tt = pd.concat([df_alagamento, one_hot_encoded], axis=1)

In [ ]:
# Atribuindo valores 1 e 0 para cada tipo de condição

boolean_columns = ["INTRANSITAVEL", "TRANSITAVEL"]
df_alagamento_tt[boolean_columns] = df_alagamento_tt[boolean_columns].astype(int)

In [ ]:
# Agrupando o DF por Subprefeitura e Data, somando os valores de alagamento

df_alagamento_tt = df_alagamento_tt.groupby(['SUB', 'DATA']).sum().reset_index()

In [ ]:
# Removendo coluna que foi desmembrada

df_alagamento_tt = df_alagamento_tt.drop(columns=['CONDICAO'])

In [ ]:
# Salvando na camada Trusted

import pandas as pd
from pandas_gbq import to_gbq

project_id = "elated-drive-432523-s4"  # Seu ID do projeto
table_id = "TRUSTED.DF_ALAGAMENTO_TT"  # Nome do dataset e tabela

to_gbq(df_alagamento_tt, table_id, project_id=project_id, if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 6820.01it/s]


# ETL Ocorrências

In [ ]:
import pandas as pd
df_ocorrencias = bf.read_gbq("elated-drive-432523-s4.RAW.DF_OCORRENCIAS")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2024-09-15 20:38:43.069721+00:00 to avoid incompatibilies with previous reads of this table. To read the latest version, set `use_cache=False` or close the current session with Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


## Tratamento de DF_OCORRENCIAS

In [ ]:
# Renomeando colunas

df_ocorrencias = df_ocorrencias.rename(columns={'data': 'DATA','ocorrencia':'OCORRENCIA'})

In [ ]:
# Removendo colunas
df_ocorrencias = df_ocorrencias.drop(columns=['subpreit','name'])

In [ ]:
# Verificando coluna DATA
df_ocorrencias['DATA']

0     2022-03-18
1     2022-03-04
2     2023-01-30
3     2023-10-05
4     2022-01-30
5     2023-12-26
6     2023-12-08
7     2023-11-04
8     2022-03-04
9     2023-01-16
10    2022-03-29
11    2024-03-19
12    2024-01-08
13    2023-12-17
14    2023-03-31
15    2022-05-18
16    2023-11-04
17    2023-02-16
18    2024-05-24
19    2023-06-05
20    2024-01-08
21    2022-03-21
22    2022-10-10
23    2023-03-09
24    2022-12-05
Name: DATA, dtype: date32[day][pyarrow]

## Criação do DataFrame final no formato TRUSTED

In [ ]:
df_ocorrencias = df_ocorrencias.to_pandas()

In [ ]:
# Separando os tipos de ocorrencia em diferentes colunas, e depois concatenando de volta ao df

one_hot_encoded = pd.get_dummies(df_ocorrencias['OCORRENCIA'])
df_ocorrencias_tt = pd.concat([df_ocorrencias, one_hot_encoded], axis=1)


In [ ]:
# Atribuindo valores 1 e 0 para cada tipo de ocorrencia

boolean_columns = ["ALAGAMENTO", "DESLIZAMENTO", "INUNDACAO", "QUEDA DE ARVORE"]
df_ocorrencias_tt[boolean_columns] = df_ocorrencias_tt[boolean_columns].astype(int)

In [ ]:
# Removendo coluna que foi desmembrada

df_ocorrencias_tt = df_ocorrencias_tt.drop(columns=['OCORRENCIA'])

In [ ]:
# Agrupando o DF por Subprefeitura e Data, somando os valores de ocorrencias

df_ocorrencias_tt = df_ocorrencias_tt.groupby(['SUB', 'DATA']).sum().reset_index()


In [ ]:
# Convertendo a DATA

df_ocorrencias_tt['DATA'] = pd.to_datetime(df_ocorrencias_tt['DATA'])

In [ ]:
# Salvando na camada Trusted

import pandas as pd
from pandas_gbq import to_gbq

project_id = "elated-drive-432523-s4"  # Seu ID do projeto
table_id = "TRUSTED.DF_OCORRENCIAS_TT"  # Nome do dataset e tabela

to_gbq(df_ocorrencias_tt, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 7384.34it/s]


# ETL Temperatura, Umidade Relativa do Ar + Pluviometria

In [ ]:
import pandas as pd
from pandas_gbq import to_gbq
import bigframes.pandas as bf

bf.options.bigquery.location = "US"
bf.options.bigquery.project = "elated-drive-432523-s4"

df_pluviometria = bf.read_gbq("elated-drive-432523-s4.RAW.DF_PLUVI")
df_temp = bf.read_gbq("elated-drive-432523-s4.RAW.DF_TEMP_UMIDADE")


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2024-09-19 00:45:43.146667+00:00 to avoid incompatibilies with previous reads of this table. To read the latest version, set `use_cache=False` or close the current session with Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2024-09-18 23:34:47.060178+00:00 to avoid incompatibilies with previous reads of this table. To read the latest version, set `use_cache=False` or close the current session with Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
print(df_pluviometria.dtypes)
print(df_temp.dtypes)


DATA                    timestamp[us][pyarrow]
IndiceDePluviometria                   Float64
SUB                            string[pyarrow]
dtype: object
DATA       timestamp[us][pyarrow]
SUB               string[pyarrow]
Tem Max                   Float64
Tem Min                   Float64
UR                        Float64
dtype: object


In [ ]:
df_pluviometria['DATA'] = pd.to_datetime(df_pluviometria['DATA'])
df_temp['Data'] = pd.to_datetime(df_temp['Data'])

Exception ignored in: <generator object RowIterator._to_page_iterable at 0x7d864268e570>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/table.py", line 1809, in _to_page_iterable
    yield from result_pages
  File "/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py", line 940, in _download_table_bqstorage
    pool.shutdown(wait=True)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 235, in shutdown
    t.join()
  File "/usr/lib/python3.10/threading.py", line 1096, in join
    self._wait_for_tstate_lock()
  File "/usr/lib/python3.10/threading.py", line 1116, in _wait_for_tstate_lock
    if lock.acquire(block, timeout):
KeyboardInterrupt: 


In [ ]:
df_pluviometria

,DATA,IndiceDePluviometria,SUB
0,2022-04-21,0.0,VP
1,2022-04-25,0.0,PA
2,2022-01-06,6.1,ST
3,2024-02-01,1.8,FO
4,2023-08-07,0.0,LA
5,2023-10-01,40.5,SM
6,2023-08-24,2.6,MB
8,2023-12-15,0.2,GU
9,2022-03-16,22.1,AF
10,2022-08-24,0.0,MG


In [ ]:
df_temp = df_temp.rename(columns={'Data': 'DATA'})
df_temp['SUB'] = df_temp['SUB'].replace({'CS-SUB': 'CS', 'CGE': 'SE'})
df_temp = df_temp[df_temp['SUB'] != 'CS-PA']
df_pluviometria = df_pluviometria[df_pluviometria['SUB'] != 'SE']
df_pluviometria['SUB'] = df_pluviometria['SUB'].replace({'Con': 'SE'})
df_pluviometria = df_pluviometria.drop(columns=['Zona', 'SubPrefeitura'])

In [ ]:
df_temp = df_temp.to_pandas()
df_pluviometria = df_pluviometria.to_pandas()

In [ ]:
df_clima = pd.merge(df_pluviometria, df_temp, on=['DATA', 'SUB'], how='outer')


In [ ]:
df_clima

,DATA,IndiceDePluviometria,SUB,Tem Max,Tem Min,UR
0,2022-01-01 00:00:00,5.5,AD,28.9,17.9,54.9
1,2022-01-01 00:00:00,2.0,AF,25.8,18.2,65.4
2,2022-01-01 00:00:00,2.2,BT,28.2,17.4,69.2
3,2022-01-01 00:00:00,3.6,CL,26.7,17.9,61.0
4,2022-01-01 00:00:00,4.3,CS,27.1,18.1,60.4
...,...,...,...,...,...,...
29983,2024-05-31 00:00:00,<NA>,SE,21.36,12.68,50.39
29984,2024-05-31 00:00:00,0.0,SM,<NA>,<NA>,<NA>
29985,2024-05-31 00:00:00,0.0,ST,20.3,11.38,53.9
29986,2024-05-31 00:00:00,0.0,VM,19.84,10.97,54.56


In [ ]:
# Salvando na camada Trusted

import pandas as pd
from pandas_gbq import to_gbq

project_id = "elated-drive-432523-s4"
table_id = "TRUSTED.DF_CLIMA_TT"

to_gbq(df_clima, table_id, project_id=project_id, if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 8289.14it/s]
